In [ ]:
import copy
import math
import matplotlib.pyplot as plt

In [ ]:
# r1: amount of asset 1
# r2: amount of asset 1
# s: pool shares
# fee: trading fee
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.00},
    "Trader":{"r1":100,"r2":100,"s":0},
    "LP":{"r1":0,"r2":0,"s":100}
}

In [ ]:
def update_pool_fees(state, fee):
    if type(fee)==float and fee >= 0 and fee < 1:
        state["AMM"]["fee"] = fee
    else:
        raise TypeError('Only floats between 0 and 1 are allowed')

In [ ]:
# state: at first genesis as defined above
# inputs = [agent_name, delta_r1]
def swapToAsset2(state,inputs):
    agent = inputs[0]
    dA1 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA2 = state["AMM"]["r2"]/(state["AMM"]["r1"]+dA1*feeFactor)*dA1*feeFactor
    if dA1>0 and state[agent]["r1"]-dA1 >= 0 :
        state["AMM"]["r1"]+=dA1
        state[agent]["r1"]-=dA1
        state["AMM"]["r2"]-=dA2
        state[agent]["r2"]+=dA2 

In [ ]:
# state: at first genesis as defined above
# inputs = [agent_name, delta_r2]
def swapToAsset1(state,inputs):
    agent = inputs[0]
    dA2 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA1 = state["AMM"]["r1"]/(state["AMM"]["r2"]+dA2*feeFactor)*dA2*feeFactor
    if dA2>0 and state[agent]["r2"]-dA2 >= 0 :
        state["AMM"]["r2"]+=dA2
        state[agent]["r2"]-=dA2
        state["AMM"]["r1"]-=dA1
        state[agent]["r1"]+=dA1 

In [ ]:
def pctToAmount(r,pct):
    if type(pct)==int and pct >= 0 and pct <= 100:
        return (r/100) * pct
    else:
        raise TypeError('Only integers between 0 and 100 are allowed')

In [ ]:
# state: at first genesis as defined above
# inputs_pct = [agent_name, delta_r1_pct]
def swapToAsset2_pct(state,inputs_pct):
    agent = inputs_pct[0]
    inputs = [
        inputs_pct[0],
        pctToAmount(state[agent]["r1"], inputs_pct[1])
    ]
    swapToAsset2(state,inputs)

In [ ]:
# state: at first genesis as defined above
# inputs_pct = [agent_name, delta_r2_pct]
def swapToAsset1_pct(state,inputs_pct):
    agent = inputs_pct[0]
    inputs = [
        inputs_pct[0],
        pctToAmount(state[agent]["r2"], inputs_pct[1])
    ]
    swapToAsset1(state,inputs)

In [ ]:
def addLiquidity(state,inputs):
    agent = inputs[0]
    R1=state["AMM"]["r1"]
    R2=state["AMM"]["r2"]
    S=state["AMM"]["s"]
    dA1=min(inputs[1],R1/R2*inputs[2])
    dA2=min(inputs[2],R2/R1*inputs[1])
    if (dA1 <= state[agent]["r1"] and dA2 <= state[agent]["r2"]) and (dA1 > 0 and dA2 > 0):
        state[agent]["r1"]-=dA1
        state[agent]["r2"]-=dA2
        state["AMM"]["r1"]+=dA1
        state["AMM"]["r2"]+=dA2
        dS = min(dA1/R1, dA2/R2) * S
        state["AMM"]["s"]+=dS
        state[agent]["s"]+=dS

In [ ]:
def removeLiquidity(state,inputs):
    dS = inputs[1]
    agent = inputs[0]
    if dS > 0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0:
        DR = (1-dS/state["AMM"]["s"])
        R1=state["AMM"]["r1"]
        R2=state["AMM"]["r2"]
        state[agent]["s"]-=dS
        state["AMM"]["r1"]=R1*DR
        state["AMM"]["r2"]=R2*DR
        state[agent]["r1"]+=R1-state["AMM"]["r1"]
        state[agent]["r2"]+=R2-state["AMM"]["r2"]
        state["AMM"]["s"]-=dS

In [ ]:
def nice_print(self):
    if type(self)==float:
        return round(self,3)
    if type(self)==int:
        return self
    if type(self)==list:
        return list(map(lambda l: print(l),self))
    if type(self)==dict:
        return dict(map(lambda kv: (kv[0], print(kv[1])),self.items()))

In [ ]:
def check_genesis_block(state):
    if state["AMM"]["fee"] < 0 or state["AMM"]["fee"] >= 1:
        return False
    shares = -state["AMM"]["s"]
    for agent in state:
        if state[agent]["r1"] < 0 or state[agent]["r2"] < 0 or state[agent]["s"] < 0:
            return False
        shares += state[agent]["s"]
    if shares != state["AMM"]["s"]:
        return False
    return True

In [ ]:
def fiat_print_holdings(state, agent = "Trader", token1 = {"name": "r1", "price": 5}, token2 = {"name": "r2", "price": 3}, ref_currency = "USD"):
    return state[agent][token1["name"]] * token1["price"] + state[agent][token2["name"]] * token2["price"]

In [ ]:
actionList = [
    [ swapToAsset2 , [  "Trader" , 50 ]],
    [ swapToAsset1 , [  "Trader" , 25 ]]
]

In [ ]:
def evolve(state, actionStack):
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

# Simple Tests

In [ ]:
evolve(genesis,actionList)
genesis

In [ ]:
update_pool_fees(genesis, 0.05)
genesis

In [ ]:
swapToAsset1_pct(genesis,["Trader", 30])
genesis

In [ ]:
check_genesis_block(genesis)

In [ ]:
fiat_print_holdings(genesis)

nice_print(genesis)

# Verifying the Digital Twin

In [ ]:
def invariant(state):
    return state["AMM"]["r1"] * state["AMM"]["r2"]

def asset1(state):
    return state["AMM"]["r1"] + state["Trader"]["r1"] + state["LP"]["r1"]

def asset2(state):
    return state["AMM"]["r2"] + state["Trader"]["r2"] + state["LP"]["r2"]

## Test 1

In [ ]:
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.0},
    "Trader":{"r1":100,"r2":100,"s":0},
    "Liquidator":{"r1":0,"r2":0,"s":100}
}

state = copy.deepcopy(genesis)

swapToAsset2(state, ["Trader",13])

invariant(genesis) == invariant(state)

## Test 2

In [ ]:
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.01},
    "Trader":{"r1":100,"r2":100,"s":0},
    "Liquidator":{"r1":0,"r2":0,"s":100}
}

state = copy.deepcopy(genesis)

swapToAsset2(state, ["Trader",13])

invariant(genesis) != invariant(state)

## Test 3

In [ ]:
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.0},
    "Trader":{"r1":100,"r2":100,"s":0},
    "LP":{"r1":0,"r2":0,"s":100}
}

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

## Test 4

In [ ]:
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.0},
    "Trader":{"r1":100,"r2":100,"s":0},
    "LP":{"r1":0,"r2":0,"s":100}
}

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

gained = state["Trader"]["r1"] - genesis["Trader"]["r1"]

swapToAsset2(state,["Trader",gained])

genesis == state

## Test 5

In [ ]:
genesis = {
    "AMM":{"r1":100,"r2":100,"s":100,"fee":0.0},
    "Trader":{"r1":100,"r2":100,"s":0},
    "LP":{"r1":0,"r2":0,"s":100}
}

state = copy.deepcopy(genesis)

addLiquidity(state, ["Trader", 30, 40])

gained_r1 = abs(state["Trader"]["r1"] - genesis["Trader"]["r1"])
gained_r2 = abs(state["Trader"]["r2"] - genesis["Trader"]["r2"])

removeLiquidity(state, ["Trader", gained_r1, gained_r2])

state == genesis

## Test 6

In [ ]:
genesis = {
    "AMM":{"r1":99,"r2":1,"s":math.sqrt(99),"fee":0.0},
    "Trader":{"r1":1,"r2":99,"s":0},
    "LP":{"r1":200,"r2":200,"s":100}
    }

# multiplication operator * on a list creates multiple repetitions of the original list
actionList= [ [swapToAsset1,["Trader",1]] ] * 99

history = evolve(genesis, actionList)

AMM_r1=[]
Trader_r1=[]
AMM_r2=[]
Trader_r2=[]
for s in history:
    AMM_r1.append(s["AMM"]["r1"])
    Trader_r1.append(s["Trader"]["r1"])
    AMM_r2.append(s["AMM"]["r2"])
    Trader_r2.append(s["Trader"]["r2"])   


plt.figure(figsize=(10,4)) 
    
plt.subplot(1, 2, 1) # one row, two columns, 1st column
plt.plot(AMM_r1,AMM_r2)
plt.title('AMM')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

plt.subplot(1, 2, 2) # one row, two columns, 2nd column
plt.plot(Trader_r1,Trader_r2)
plt.title('Trader')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

## Test 7

In [ ]:
actionList.insert(0,[addLiquidity,["LP",100,100]])

genesis = {
    "AMM":{"r1":99,"r2":1,"s":math.sqrt(99),"fee":0.0},
    "Trader":{"r1":1,"r2":99,"s":0},
    "LP":{"r1":200,"r2":200,"s":100}
    }

history = evolve(genesis, actionList)

AMM_2r1=[]
Trader_2r1=[]
AMM_2r2=[]
Trader_2r2=[]
for s in history:
    AMM_2r1.append(s["AMM"]["r1"])
    Trader_2r1.append(s["Trader"]["r1"])
    AMM_2r2.append(s["AMM"]["r2"])
    Trader_2r2.append(s["Trader"]["r2"])   
    
plt.figure(figsize=(4,4)) 
plt.plot(AMM_r1,AMM_r2)
plt.plot(AMM_2r1,AMM_2r2)
plt.title('AMM')
plt.xlim([0, 99])
plt.ylim([0, 99])
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")